# Import

In [50]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
from sklearn.datasets import make_classification # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore
from sklearn.metrics import accuracy_score # type: ignore
import tensorflow as tf # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.calibration import LabelEncoder # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error # type: ignore
from sklearn.ensemble import RandomForestRegressor # type: ignore
from sklearn.tree import DecisionTreeRegressor # type: ignore
import seaborn as sns # type: ignore

path

In [51]:
path = "C:/Users/Alessio/Desktop/Corso IA/Machine Learning/Esercitazione Neural Network/Dataset"

# Pulizia del dataset

In [52]:
df = pd.read_csv(path + "/AmesHousing.csv")

In [53]:
df = pd.DataFrame(df)

In [ ]:
df.info()

In [ ]:
df.head()

Fillo le colonne NaN

In [56]:
df_fill = df.copy()

for col in df_fill.columns:
    if df_fill[col].isnull().sum() > 0:
        if df_fill[col].dtype == "object":
            df_fill[col] = df_fill[col].fillna("")
        else:
            df_fill[col] = df_fill[col].fillna(0)

In [ ]:
df_fill.info()

In [ ]:
missing_values = df_fill.isnull().sum()
print("Valori mancanti per colonna:")
print(missing_values)

Controllo dimensioni del dataset

In [ ]:
print(f"Dimensioni del dataset: {df.shape}")
print(f"Totale righe: {df.shape[0]}")
print(f"Totale colonne: {df.shape[1]}")

Encoding

In [60]:
def encode_non_numeric_columns(df):
    #Crea un oggetto LabelEncoder
    le = LabelEncoder()
    
    #Itera sulle colonne del DataFrame
    for col in df.columns:
        #Verifica se la colonna non è numerica
        if df[col].dtype == 'object':  # Puoi anche usare `df[col].apply(type).eq(str).any()`
            #Applica il LabelEncoder solo se la colonna è di tipo 'object' (non numerica)
            df[col] = le.fit_transform(df[col])
    
    return df

In [61]:
df_encoded = encode_non_numeric_columns(df_fill)

In [ ]:
df_encoded.head()

In [ ]:
df_encoded.info()

Dati scalati

In [64]:
scaler = StandardScaler()
df_encoded[df_encoded.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(df_fill.select_dtypes(include=['float64', 'int64']))

Split dataset Train Fit

In [65]:
X = df_encoded.drop('SalePrice', axis=1)  
y = df_encoded['SalePrice'] 

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Rete Neurale

In [ ]:
model_custom = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_val.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)
])

In [ ]:
model_custom.summary()

In [81]:
model_custom.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

Funzione di callback

In [82]:
early_stopping = EarlyStopping(
    monitor='loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

In [ ]:
history = model_custom.fit(X_train_val, y_train_val, epochs=100, batch_size=32, verbose=1, validation_data=(X_val, y_val), callbacks=[early_stopping])

Visualizzazione MSE e Loss insieme alla validation

In [ ]:
#Grafico dell'andamento della loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss durante il training')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()

#Grafico dell'andamento dell'MSE
plt.subplot(1, 2, 2)
plt.plot(history.history['mean_squared_error'], label='Train MSE')
plt.plot(history.history['val_mean_squared_error'], label='Validation MSE')
plt.title('MSE durante il training')
plt.xlabel('Epoche')
plt.ylabel('Mean Squared Error')
plt.legend()

plt.tight_layout()
plt.show()

Valutazione sul test MAE RMSE

In [ ]:
y_pred = model_custom.predict(X_test)

# Calcola MAE
mae = mean_absolute_error(y_test, y_pred)

# Calcola RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

Comparazione tra dati predetti e dati effettivi nel test

In [87]:
y_pred = y_pred.flatten()

In [ ]:
correlation = np.corrcoef(y_test, y_pred)[0, 1]
print(f"Correlazione tra i prezzi reali e quelli predetti: {correlation:.4f}")

Correlation matrix per visualizzare il peso effettivo delle features

In [ ]:
correlation_matrix = df_fill.corr()

for col in df_fill.columns:
    print(f"Correlazione tra SalePrice e {col}: {correlation_matrix['SalePrice'][col]:.4f}")

#I valori negativi non influiscono nella predizione, pertanto sono inutili

# Modello semplice di regressione

In [90]:
#Random forest regressor
random_forest_regr = RandomForestRegressor(
    n_estimators = 10,
    random_state = 42
)

In [ ]:
random_forest_regr.fit(X_train,y_train)

In [92]:
y_pred_forest_regr = random_forest_regr.predict(X_test)

In [ ]:
regressor = DecisionTreeRegressor(random_state=42)

#Addestra il modello
regressor.fit(X_train, y_train)

#Effettua previsioni sui dati di test
y_pred = regressor.predict(X_test)

#Valuta le performance del modello
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

In [ ]:
#Calcola MAE
mae = mean_absolute_error(y_test, y_pred_forest_regr)
#Calcola RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_forest_regr))

print(f"MAE random forest: {mae}")
print(f"RMSE random forest: {rmse}")

y_pred = model_custom.predict(X_test)

#Calcola MAE
mae = mean_absolute_error(y_test, y_pred)
#Calcola RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE NN {mae}")
print(f"RMSE NN: {rmse}")

# **Conclusioni Parte 1**

In questo esperimento, sono stati utilizzati tre modelli di apprendimento automatico per prevedere il valore della variabile SalePrice in base a diverse caratteristiche del dataset. I modelli scelti per questa analisi sono:

- Random Forest
- Rete Neurale (NN)
- Decision Tree Regressor

Tutti i modelli sono stati addestrati e valutati sui dati di test, con i seguenti risultati.

**Risultati per Random Forest**
- MAE (Errore Assoluto Medio): 0.2238
- RMSE (Errore Quadratico Medio): 0.3579

**Risultati per la Rete Neurale (NN)**
- MAE (Errore Assoluto Medio): 0.1922
- RMSE (Errore Quadratico Medio): 0.3652
- Abbiamo provato diverse complessità per la rete neurale, a complessità molto elevate (512/1024 primo layer) si rischia un overfit tuttavia aumentando di complessità la precisione aumenta

**Risultati per Decision Tree**
- MAE (Errore Assoluto Medio): 0.3185
- RMSE (Errore Quadratico Medio): 0.4768

### **Analisi dei Risultati**

**1.Errore Assoluto Medio (MAE):**
- La Rete Neurale (NN) ha mostrato un errore assoluto medio (MAE) inferiore rispetto alla Random Forest (0.1922 vs. 0.2238).
Ciò indica che la Rete Neurale è più precisa nel predire il valore assoluto del prezzo di vendita, mostrando una leggera superiorità in termini di accuratezza.

**2.Errore Quadratico Medio (RMSE):**
- Sebbene la Rete Neurale (NN) abbia un errore assoluto medio più basso, l'RMSE della Rete Neurale risulta essere leggermente superiore rispetto alla Random Forest (0.3652 vs. 0.3579).
- Questo suggerisce che la Random Forest ha una prestazione migliore in termini di riduzione dell'errore quadratico medio, gestendo in modo più efficace gli outliers o valori estremi.

**3.Decision Tree:**
- Il Decision Tree, pur essendo un modello semplice e facile da interpretare, ha mostrato i peggiori risultati tra i tre modelli, con il più alto MAE e RMSE.
- Questo potrebbe essere dovuto alla sua sensibilità agli overfitting, soprattutto se non regolarizzato correttamente, e alla sua limitata capacità di generalizzare rispetto ai modelli più complessi come la Random Forest e la Rete Neurale.

**Considerazioni Finali:**
Entrambi i modelli hanno funzionato bene, ma la Rete Neurale ha avuto un errore assoluto medio (MAE) inferiore, il che indica che è stata leggermente più accurata nelle previsioni generali.
D'altra parte, la Random Forest ha mostrato un errore quadratico medio (RMSE) inferiore, il che la rende più adatta per la gestione dei valori anomali o per la riduzione degli errori estremi.

**Approccio Migliore in Funzione degli Obiettivi:**
Se l'obiettivo principale è minimizzare l'errore assoluto medio (MAE) e ottenere previsioni più precise in generale, la Rete Neurale ha un leggero vantaggio.
Se invece si desidera ridurre l'errore quadratico medio (RMSE) e gestire meglio i valori anomali, la Random Forest potrebbe essere la scelta migliore.

# **Conclusione Finale**
In sintesi, tutti e tre i modelli hanno mostrato buone prestazioni, ma la Rete Neurale si distingue per la sua precisione nelle previsioni, mentre la Random Forest è preferibile in scenari dove è necessario gestire gli outliers e ridurre l'impatto degli errori quadrati. Il Decision Tree, pur essendo utile in situazioni in cui la semplicità e l'interpretabilità sono prioritari, ha mostrato prestazioni inferiori in questo esperimento.
La scelta del modello dipenderà quindi dalle specifiche necessità del problema e dagli obiettivi da raggiungere, come la gestione dei valori estremi, la precisione nelle previsioni o la semplicità del modello.